The ground Hatree-Fock state is when 1up and 1down orbitals (molecular orbitals) are occupied. This is (1, 1, 0, 0) in fermion occupation basis. In Bravyi-Kitaev basis, this is (1, 0, 0, 0), so just X(0). 

In [ ]:
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.utils import hermitian_conjugated
from openfermion.utils import uccsd_generator
import numpy as np

from pyquil.paulis import PauliSum
from pyquil.api import WavefunctionSimulator
from scipy.optimize import minimize
from pyquil import Program
from pyquil.gates import *

from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli
from forestopenfermion import exponentiate

import numpy as np
import functools

from openfermion.hamiltonians import MolecularData, load_molecular_hamiltonian

import time

In [2]:
from pyquil import get_qc
qc = get_qc('Aspen-4-16Q-A-qvm')

In [3]:
numQubit = 4
trials = 100
class Track:
    def __init__(self):
        self.cnt = 0
        
def build_ansatz(params):
    # Reference state
    a = None
    a = Program()

    single_amp = []
    double_amp = []

    # occupied: 0,1 unoccupied: 2,3
    single_amp.append([[2,0], params[0]])
    single_amp.append([[3,1], params[1]])

    if degree == 2:
        double_amp.append([[2,0,3,1], params[2]])

    ucc_gen = uccsd_generator(single_amp, double_amp)
    ucc_gen_qubitop = bravyi_kitaev(ucc_gen)
    try:
        ucc_program = exponentiate(ucc_gen_qubitop/(-1j))
    except:
        ucc_program = I(0)
    a += ucc_program
    return a

def solve_vqe_ucc_h2(hamiltonian: PauliSum, numLayer) -> float:
    # Construct a variational quantum eigensolver solution to find the lowest
    # eigenvalue of the given hamiltonian
    
    numParams = 0
    if degree == 1:
        numParams = 2
    elif degree == 2:
        numParams = 3
    params_init = np.random.rand(numParams)*0.5-0.25
    #params_init = [7.25536731e-07,  1.04666900e-06, -1.04874604e-01]
    
    track = Track()

    def inc(t, xk):
        t.cnt += 1

    def ansatz_energy(t, params):
        t.cnt += 1
        energy = 0
        ansatz = None
        ansatz = Program()
        ansatz += build_ansatz(params)
        
        energy = 0
        for term in hamiltonian:
            p = None
            p = Program()
            p += X(0)
            p += ansatz

            coeff = term.coefficient.real    # coefficient is real number in complex format
            opset =  term.operations_as_set()
            qubits_meas = []
            for op in opset:
                if op[1]=='X':
                    p += H(op[0])
                elif op[1]=='Y':
                    p += RZ(np.pi/2, op[0])
                    p += RY(np.pi/2, op[0])
                    p += RZ(-np.pi/2, op[0])
                elif op[1]=='Z':
                    pass
                qubits_meas.append(op[0])
  
            result = qc.run_and_measure(p, trials=100)
            # measure the expectation value of eigenvalue in Z..Z basis. (number of 1 - number of -1)/trials
            eigval = 0
            for i in range(trials):
                tmp = 1
                for j in qubits_meas:
                    if result[j][i] == 1:
                        tmp = -tmp
                eigval += tmp
            energy += coeff*float(eigval)/trials
            
        print(energy)
        return energy

    option = {}
    option['ftol'] = 0.1
    params_answer = minimize(functools.partial(ansatz_energy, track), params_init, method='L-BFGS-B', options = option).x
    return ansatz_energy(track, params_answer), track.cnt

In [4]:
def get_ground_energy_ucc_h2(interaction_hamil, degree):
    fermionop_hamil = FermionOperator()
    for key in interaction_hamil:
        value = interaction_hamil[key]
        fermionop_hamil += FermionOperator(term=key, coefficient=value)
        
    qubitop_hamil = bravyi_kitaev(fermionop_hamil)
    pauliop_hamil = qubitop_to_pyquilpauli(qubitop_hamil)

    return solve_vqe_ucc_h2(pauliop_hamil, degree)


In [5]:
basis = 'sto-3g'
multiplicity = 1  # 2S+1
charge = 0

import matplotlib.pyplot as plt
%matplotlib inline

bond_lengths = np.linspace(0.3, 2.4, 8)
vqe_ground_energies = []
vqe_iters = []
vqe_times = []

degree = 1

current_time = time.time()
print(current_time)

for bond_length in bond_lengths:
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    description = str(round(bond_length, 2))
    h2_interaction_hamil = load_molecular_hamiltonian(geometry,
        basis,
        multiplicity,
        description,
        n_active_electrons=None,
        n_active_orbitals=None)


    ge, cnt = get_ground_energy_ucc_h2(h2_interaction_hamil, degree)
    timelapse = time.time() - current_time
    vqe_times.append(timelapse)
    print('bond length: ', round(bond_length, 2), ' ground state energy: ', ge, ' iteration: ', cnt, 'time: ', timelapse)
    
    vqe_ground_energies.append(ge)
    vqe_iters.append(cnt)
    
    current_time = time.time()
        
print(vqe_ground_energies)
print(np.mean(vqe_iters))
print(np.std(vqe_iters))
print(np.mean(vqe_times))


1559732257.8225386
-0.47146257305824313
-0.4747158086698258
-0.4124908068423184
0.40903354091548755
0.4688368746008666
0.4131872798807182
-0.45246096479940867
-0.42144005708037663
-0.4580459969469484
-0.5301988900900902
-0.4952012878435208
-0.5287528628122735
-0.5303294620999321


KeyboardInterrupt: 

In [ ]:
# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1

vqe_qvm_deg1 = np.array([-0.4624058470678082, -1.015415011064866, -1.078574021527077, -1.0080117573300524, -0.8935215369858669, -0.8175487109835171, -0.7321655888885132, -0.7673884787306767])
vqe_qvm_deg1_err = vqe_qvm_deg1*0.1
    
# Generate molecule at different bond lengths.
fci_energies = []
bond_lengths = []
for bond_length in np.linspace(0.3, 2.5, 23):
    bond_lengths += [bond_length]
    description = str(round(bond_length,2))
#    print(description)
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    molecule = MolecularData(
        geometry, basis, multiplicity, description=description)
    
    # Load data.
    molecule.load()
    fci_energies += [molecule.fci_energy]

fig, ax = plt.subplots(figsize=(16,10))
ax.errorbar(np.linspace(0.3, 2.4, 8), vqe_qvm_deg1, vqe_qvm_deg1_err, color='g', marker='o', linewidth=1, markersize=5, label='degree=1')
plt.plot(bond_lengths, fci_energies, 'r-', linewidth=1, markersize=1, label='exact solution')
plt.legend()
plt.ylabel('Ground state energy in Hartree', size=15)
plt.xlabel('Atomic distance in Angstrom', size=15)
plt.savefig('PLOT5.pdf', bbox_inches='tight')
ax.tick_params(labelsize = 12)
plt.show()



In [ ]:
fci_energies_reduced = []
for i in range(8):
    fci_energies_reduced.append(fci_energies[3*i])
fci_energies_reduced = np.array(fci_energies_reduced)
print(np.mean(1-np.abs((vqe_qvm_deg1-fci_energies_reduced)/fci_energies_reduced)))
